In [ ]:
! pip install transformers

%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 645kB 5.0MB/s 
     |████████████████████████████████| 1.1MB 13.5MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
     |████████████████████████████████| 3.8MB 17.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f6c3dbe4904d7527ee015ff49a856a50b3b0a773c0028fb3f394cf28c37faf29
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
2.2.0


## Fine-tuning BERT
- BERT Pre-Training phase: Masked Language Model(MLM) and Next Sentence Prediction(NSP) 

- Then, finetune all of the weights in the pretrained model for a task of choice.

- Source: https://arxiv.org/pdf/1810.04805.pdf

<img src="https://i.imgur.com/fKNAhKH.png" alt=" " width="75%" height="75%">



##  QuestionAnswering Model (inbuilt)
- Pretraining(NSP, MLM) + SQuAD Dataset trained model
- Extractive Model
- SQuAD: https://rajpurkar.github.io/SQuAD-explorer/

- "SQuAD2.0 combines the 100,000 questions in SQuAD1.1 with over 50,000 unanswerable questions written adversarially by crowdworkers to look similar to answerable ones. To do well on SQuAD2.0, systems must not only answer questions when possible, but also determine when no answer is supported by the paragraph and abstain from answering."



In [ ]:
# Using HuggingFace's QuestionAnsweringModel (inbuilt)

import tensorflow as tf

# Refer: https://huggingface.co/transformers/model_doc/bert.html

from transformers import BertTokenizer, TFBertForQuestionAnswering 

modelName = 'bert-large-uncased-whole-word-masking-finetuned-squad' # https://huggingface.co/transformers/pretrained_models.html
 
tokenizer = BertTokenizer.from_pretrained(modelName)
model = TFBertForQuestionAnswering.from_pretrained(modelName)

print(model)

In [ ]:
# Text+Q&A from SQuAD Test data: https://rajpurkar.github.io/SQuAD-explorer/explore/v2.0/dev/Amazon_rainforest.html?model=nlnet%20(single%20model)%20(Microsoft%20Research%20Asia)&version=v2.0
text = r"""The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species."""
question =r"""What percentage does the Amazon represents in rainforests on the planet?"""

#question = r"""How many nations contain "Amazonas" in their names?"""


#input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
input_text =  question + " [SEP] " + text 
input_ids = tokenizer.encode(input_text)

print(len(input_ids))
print(tokenizer.decode(input_ids))

input = tf.constant(input_ids)[None, :]  # Batch size 1 


250
[CLS] what percentage does the amazon represents in rainforests on the planet? [SEP] the amazon rainforest ( portuguese : floresta amazonica or amazonia ; spanish : selva amazonica, amazonia or usually amazonia ; french : foret amazonienne ; dutch : amazoneregenwoud ), also known in english as amazonia or the amazon jungle, is a moist broadleaf forest that covers most of the amazon basin of south america. this basin encompasses 7, 000, 000 square kilometres ( 2, 700, 000 sq mi ), of which 5, 500, 000 square kilometres ( 2, 100, 000 sq mi ) are covered by the rainforest. this region includes territory belonging to nine nations. the majority of the forest is contained within brazil, with 60 % of the rainforest, followed by peru with 13 %, colombia with 10 %, and with minor amounts in venezuela, ecuador, bolivia, guyana, suriname and french guiana. states or departments in four nations contain " amazonas " in their names. the amazon represents over half of the planet's remaining rainf

In [ ]:
# Source: Modified PyTorch code from https://www.kaggle.com/c/tensorflow2-question-answering/discussion/123434

#[SEP] =  102 (token_id)
# token_type_ids has 0 corresponding to the qeustion part of the input_text and 1 corresponding to the 
# text/part 

token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

print(token_type_ids)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
answer=model(input, token_type_ids = tf.convert_to_tensor([token_type_ids]))

print(type(answer))
print(len(answer))

# https://huggingface.co/transformers/model_doc/distilbert.html#tfdistilbertforquestionanswering

startScores, endScores = answer
print(startScores.shape)
print(endScores.shape)
print(type(startScores))




<class 'tuple'>
2
(1, 250)
(1, 250)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [ ]:
input_tokens = tokenizer.convert_ids_to_tokens(input_ids)

print(input_tokens)

['[CLS]', 'what', 'percentage', 'does', 'the', 'amazon', 'represents', 'in', 'rainforest', '##s', 'on', 'the', 'planet', '?', '[SEP]', 'the', 'amazon', 'rainforest', '(', 'portuguese', ':', 'flores', '##ta', 'amazon', '##ica', 'or', 'amazon', '##ia', ';', 'spanish', ':', 'se', '##lva', 'amazon', '##ica', ',', 'amazon', '##ia', 'or', 'usually', 'amazon', '##ia', ';', 'french', ':', 'fore', '##t', 'amazon', '##ien', '##ne', ';', 'dutch', ':', 'amazon', '##ere', '##gen', '##wo', '##ud', ')', ',', 'also', 'known', 'in', 'english', 'as', 'amazon', '##ia', 'or', 'the', 'amazon', 'jungle', ',', 'is', 'a', 'moist', 'broad', '##leaf', 'forest', 'that', 'covers', 'most', 'of', 'the', 'amazon', 'basin', 'of', 'south', 'america', '.', 'this', 'basin', 'encompasses', '7', ',', '000', ',', '000', 'square', 'kilometres', '(', '2', ',', '700', ',', '000', 'sq', 'mi', ')', ',', 'of', 'which', '5', ',', '500', ',', '000', 'square', 'kilometres', '(', '2', ',', '100', ',', '000', 'sq', 'mi', ')', 'are', 

In [ ]:
print(startScores)


tf.Tensor(
[[-6.047485   -4.656026   -7.571985   -7.812991   -7.006605   -7.8947506
  -8.126448   -7.540066   -8.892705   -9.418996   -8.942824   -8.615166
  -9.496941   -9.665026   -6.047458   -6.1517916  -3.5666323  -7.074654
  -8.149385   -6.5617337  -8.559914   -7.1163793  -8.553459   -7.5595994
  -8.405014   -8.682979   -6.7906084  -8.405868   -8.498199   -6.235168
  -8.774738   -7.628984   -8.575293   -7.887277   -8.571484   -8.881729
  -7.359875   -8.744015   -8.5534725  -6.743694   -7.268455   -8.534403
  -8.5651     -6.48624    -8.576065   -6.986378   -8.542225   -7.4793925
  -8.85868    -8.480019   -8.771891   -6.91767    -8.668295   -7.195727
  -8.835159   -8.785324   -8.765894   -8.421247   -7.56776    -7.990117
  -7.5482945  -7.821804   -8.122393   -7.256473   -8.40498    -6.5453873
  -8.211745   -8.5522     -7.2628684  -6.084314   -7.759271   -8.038822
  -7.1379924  -7.3019333  -7.178515   -7.6427264  -8.35898    -7.0667434
  -8.150645   -7.197267   -6.8137436  -8.397957 

In [ ]:
print(tf.math.argmax(startScores[0],0)) # https://www.tensorflow.org/api_docs/python/tf/math/argmax
    

tf.Tensor(207, shape=(), dtype=int64)


In [ ]:
print((tf.math.argmax(startScores[0],0)).numpy())

207


In [ ]:
startIdx = tf.math.argmax(startScores[0],0).numpy()
endIdx = tf.math.argmax(endScores[0],0).numpy()+1
print(startIdx,endIdx)

207 209


In [ ]:
print(" ".join(input_tokens[startIdx:endIdx]))

over half


## Additional Resources
- Mobile BERT QA System (code in pure TF/Keras):  https://www.tensorflow.org/lite/models/bert_qa/overview

- https://towardsdatascience.com/testing-bert-based-question-answering-on-coronavirus-articles-13623637a4ff

- Finetune on custom-data: https://huggingface.co/transformers/examples.html#squad